In [28]:
import numpy as np
import pandas as pd
import matplotlib as plt
from ucimlrepo import fetch_ucirepo 

Implementação Perceptron Clássico

In [39]:
def load_breast_cancer():
        breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

        X = breast_cancer_wisconsin_diagnostic.data.features.to_numpy()
        y = breast_cancer_wisconsin_diagnostic.data.targets.iloc[:, 0].to_numpy()

        y = np.where(y == "M", 1, -1)

        return X, y


Pre-Processamento dos Datasets

In [64]:
def stratified_kfold_standardized(X, y, k=5, shuffle=True, random_state=None):
    """
    Gera folds estratificados e padronizados (z-score) para cada iteração.
    Retorna: X_train_scaled, X_test_scaled, y_train, y_test
    """
    np.random.seed(random_state)
    classes = np.unique(y)
    
    # separar índices por classe
    class_indices = [np.where(y == c)[0] for c in classes]
    
    # embaralhar cada classe
    if shuffle:
        for idx in class_indices:
            np.random.shuffle(idx)
    
    # dividir cada classe em k folds
    folds = [[] for _ in range(k)]
    for idx in class_indices:
        split = np.array_split(idx, k)
        for i in range(k):
            folds[i].extend(split[i])
    
    # gerar folds de treino/teste
    for i in range(k):
        test_idx = np.array(folds[i])
        train_idx = np.array([idx for j in range(k) if j != i for idx in folds[j]])
        
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # padronização
        mean = X_train.mean(axis=0)
        std = X_train.std(axis=0)
        std[std == 0] = 1  # evitar divisão por zero
        X_train_scaled = (X_train - mean) / std
        X_test_scaled = (X_test - mean) / std
        
        yield X_train_scaled, X_test_scaled, y_train, y_test


def z_score(X_train, X_test):
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    std[std == 0] = 1
    X_train_scaled = (X_train - mean) / std
    X_test_scaled = (X_test - mean) / std
    return X_train_scaled, X_test_scaled        



Implementação Perceptron Clássico

In [66]:
class Perceptron:
    def __init__(self, epochs=100, eta=0.1):
        self.epochs = epochs
        self.eta = eta
        self.w = None

    def fit(self, X, y):
        n, d = X.shape
        self.w = np.zeros(d + 1)  

        for epoch in range(self.epochs):
            errors = 0
            indices = np.arange(n)
            np.random.shuffle(indices)  

            for i in indices:
                x_i = np.append(X[i], 1)  # adiciona bias
                if y[i] * np.dot(self.w, x_i) <= 0:
                    self.w += self.eta * y[i] * x_i
                    errors += 1

            if errors == 0:
                print(f"Convergência na época {epoch+1}")
                break

    def decision_function(self, X):
        X_bias = np.c_[X, np.ones(X.shape[0])]
        return X_bias @ self.w
    
    def predict(self, X):
        return np.sign(self.decision_function(X))

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)



In [69]:
def compute_macro_f1(y_true, y_pred, n_classes=None):
    """
    Calcula o Macro-F1 manualmente a partir dos rótulos verdadeiros e previstos.
    
    y_true, y_pred: arrays de rótulos (inteiros, ex: 0 e 1)
    n_classes: número de classes. Se None, inferido de y_true.
    """
    if n_classes is None:
        n_classes = len(np.unique(np.concatenate([y_true, y_pred])))
    
    f1_scores = []
    
    for i in range(n_classes):
        tp = np.sum((y_true == i) & (y_pred == i))
        fp = np.sum((y_true != i) & (y_pred == i))
        fn = np.sum((y_true == i) & (y_pred != i))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

In [70]:
def run_stratified_cross_validation():
    X, y = load_breast_cancer()
    
    param_grid = {
        "epochs": [500, 1000],
        "learning_rate": [0.001, 0.01, 0.1]
    }
    
    k_folds = 5
    best_result = None
    results = []
    
    for epochs in param_grid["epochs"]:
        for learning_rate in param_grid["learning_rate"]:
            
            accuracies_test = []
            
            for fold, (X_train_scaled, X_test_scaled, y_train, y_test) in enumerate(
                stratified_kfold_standardized(X, y, k=k_folds, random_state=42)
            ):
                model = Perceptron(epochs=epochs, eta=learning_rate)
                model.fit(X_train_scaled, y_train)
                
                test_accuracy = model.accuracy(X_test_scaled, y_test)
                accuracies_test.append(test_accuracy)
            
            mean_test = np.mean(accuracies_test)
            std_test = np.std(accuracies_test)
            
            print(f"epochs={epochs}, lr={learning_rate:.3f} -> "
                  f"Acurácia TESTE: {mean_test:.4f} ± {std_test:.4f}")
            
            results.append(((epochs, learning_rate), mean_test, std_test))
            
            if best_result is None or mean_test > best_result[1]:
                best_result = ((epochs, learning_rate), mean_test, std_test)
    
    best_params, best_mean, best_std = best_result
    print(f"\nMelhor configuração: epochs={best_params[0]}, lr={best_params[1]} "
          f"com acurácia {best_mean:.4f} ± {best_std:.4f}")

    y_pred = model.predict(X_test_scaled)
    macro_f1 = compute_macro_f1(y_test, y_pred)
    print(f"Macro-F1: {macro_f1:.4f}")

    return best_result





if __name__ == "__main__":
    best_params = run_stratified_cross_validation()



[285 169]
Convergência na época 410
[285 169]
Convergência na época 51
[286 170]
[286 170]
[286 170]
epochs=500, lr=0.001 -> Acurácia TESTE: 0.9439 ± 0.0209
[285 169]
Convergência na época 410
[285 169]
Convergência na época 51
[286 170]
[286 170]
[286 170]
epochs=500, lr=0.010 -> Acurácia TESTE: 0.9439 ± 0.0209
[285 169]
Convergência na época 410
[285 169]
Convergência na época 51
[286 170]
[286 170]
[286 170]
epochs=500, lr=0.100 -> Acurácia TESTE: 0.9439 ± 0.0209
[285 169]
Convergência na época 410
[285 169]
Convergência na época 51
[286 170]
[286 170]
[286 170]
Convergência na época 902
epochs=1000, lr=0.001 -> Acurácia TESTE: 0.9439 ± 0.0177
[285 169]
Convergência na época 410
[285 169]
Convergência na época 51
[286 170]
[286 170]
[286 170]
Convergência na época 902
epochs=1000, lr=0.010 -> Acurácia TESTE: 0.9439 ± 0.0177
[285 169]
Convergência na época 410
[285 169]
Convergência na época 51
[286 170]
[286 170]
[286 170]
Convergência na época 902
epochs=1000, lr=0.100 -> Acurácia 